# Projet de Bio informatique

In [2]:
import numpy as np
import pandas
from sklearn import datasets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import networkx as nx

## 1. Import des données

Import de la base de données non condensée.

In [55]:
cell_cycle = pandas.read_csv('exemplesCSV/cellCycle.csv')
print(cell_cycle.shape)

(100800, 28)


In [56]:
cell_cycle.head(10)

,valid,K_sk,K_sk:a_ini_sk,K_sk:sk_act1,K_sk:a_ini_sk:sk_act1,K_a,K_a:b_ini,K_a:en_ini,K_a:sk_act2,K_a:b_ini:en_ini,...,K_ep:b_act,K_en,K_en:a_ini_en,K_en:ep_act,K_en:sk_ini,K_en:a_ini_en:ep_act,K_en:ep_act:sk_ini,K_en:a_ini_en:sk_ini,K_en:a_ini_en:ep_act:sk_ini,Error explanation
0,OK,0,1,0,2,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,NaN
1,OK,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
2,OK,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
3,OK,0,1,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
4,OK,0,0,1,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
5,OK,0,0,0,2,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
6,OK,0,1,0,2,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
7,OK,0,0,1,2,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
8,OK,0,0,2,2,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,NaN
9,OK,0,1,0,2,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,NaN


On va enlever la colonne ErrorExplanation qui ne contient rien.

In [62]:
cell_cycle = cell_cycle.drop(['Error explanation'],axis=1);

In [63]:
print(cell_cycle.shape)

(100800, 27)


On voit que la base contient 100800 données. On va donc devoir trouver un moyen de toutes les traiter.

On va d'abord regarder le nombre de données du fichier condensé.

In [65]:
cell_cycle_condense = pandas.read_csv('exemplesCSV/cellCycle-MDD.csv')
print(cell_cycle_condense.shape)

(3482, 27)


Dans cette base de données il y a 3483 données.

In [66]:
cell_cycle_condense.head(10)

,valid,K_sk,K_sk:a_ini_sk,K_sk:sk_act1,K_sk:a_ini_sk:sk_act1,K_a,K_a:b_ini,K_a:en_ini,K_a:sk_act2,K_a:b_ini:en_ini,...,K_ep,K_ep:b_act,K_en,K_en:a_ini_en,K_en:ep_act,K_en:sk_ini,K_en:a_ini_en:ep_act,K_en:ep_act:sk_ini,K_en:a_ini_en:sk_ini,K_en:a_ini_en:ep_act:sk_ini
0,OK,0,0,0,0,0,0,0,0,0,...,0,0..1,0,0,0,0,0..1,0..1,1,1
1,OK,0,0,0,0,0,0,0,0,0,...,0,0..1,0,0,0,1,0..1,1,1,1
2,OK,0,0,0,0,0,0,0,0,0,...,0,0..1,0,0,1,0..1,1,1,1,1
3,OK,0,0,0,0,0,0,0,0,0,...,0,0..1,0,1,0,0,1,0..1,1,1
4,OK,0,0,0,0,0,0,0,0,0,...,0,0..1,0,1,0,1,1,1,1,1
5,OK,0,0,0,0,0,0,0,0,0,...,0,0..1,0,1,1,0..1,1,1,1,1
6,OK,0,0,0,0,0,0,0,0,0,...,0,0..1,1,1,1,1,1,1,1,1
7,OK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0..1,0..1,1,1
8,OK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0..1,1,1,1
9,OK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0..1,1,1,1,1
